# Módulo 4: Modelo de clasificación QSAR (paso a paso)

En este módulo desarrollaremos nuestro primer modelo QSAR, concretamente un modelo para la toxicidad aguda en lombrices de tierra. Dado que desarrollar un modelo QSAR implica un flujo de trabajo con varios pasos y este es tu primer modelo, este módulo será extenso, ya que exploraremos cada paso cuidadosamente. Por ello, el flujo de trabajo se divide en diferentes lecciones, y la práctica en Python correspondiente está separada en distintos archivos Jupyter Notebook. Como recordatorio, en este curso el flujo de trabajo para desarrollar un modelo QSAR se divide en las siguientes partes:

- Parte 1: Obtención y depuración de datos
- Parte 2: Cálculo de descriptores moleculares
- Parte 3: División entre entrenamiento y prueba, y estandarización
- Parte 4: Selección de descriptores
- Parte 5: Desarrollo y optimización del modelo
- Parte 6: Predicción y dominio de aplicabilidad

# Parte 1: Obtención y depuración de datos

En esta lección aprenderemos cómo preprocesar la base de datos para obtener un conjunto adecuado de moléculas con las que construir un modelo.

Primero, leeremos la base de datos, seleccionaremos qué datos queremos usar en nuestro análisis y, si es necesario, transformaremos los valores del endpoint para tener un conjunto de datos coherente.

Después, llevaremos a cabo un procedimiento de depuración para eliminar todas las moléculas que estén descritas con un SMILES incorrecto, que no sean adecuadas para nuestro modelo o que estén duplicadas.

Para comenzar, vamos a ejecutar una celda en el notebook para importar los módulos básicos que utilizaremos en esta lección. Esto puede hacerse al principio si ya sabes qué vas a usar, o justo antes de utilizar un módulo nuevo.

In [1]:
#Importa los paquetes requeridos
import pandas as pd   #Importa la librería pandas como pd
from rdkit import Chem  #Importa la libraría Chem de rdkit


## Sección 1. Lectura de un conjunto de datos de moléculas y preparación de un conjunto de datos para el modelado

El primer paso para crear un modelo QSAR es obtener y seleccionar los datos que se van a utilizar. Compilar o encontrar una buena base de datos para tu modelo es muy importante.

La forma exacta de acceder a los datos dependerá de su fuente y del formato de la base de datos. Debemos considerar cuestiones como:

- ¿Podemos descargarla directamente o necesitamos una API?

- ¿Tenemos un solo archivo o necesitamos combinar varios?

- ¿Cuál es la extensión del archivo?

- ¿Cómo están representadas las moléculas?

- ¿Disponemos de campos de datos adicionales con información útil?


Los datos pueden encontrarse en múltiples formatos, pero es muy común poder descargar bases de datos de compuestos en forma de una o más tablas. Un formato de archivo muy habitual para tablas es el de valores separados por comas (**CSV**) (consulta este enlace https://docs.fileformat.com/spreadsheet/csv/ para más detalles). Este es el formato que utilizaremos a lo largo del curso.

Observa cómo usamos la celda de abajo para leer el archivo **Earthworm_acute_toxicity_raw_data.csv** desde la carpeta en la que estamos trabajando. En este caso, abriremos la base de datos como un DataFrame de Pandas utilizando el método `read_csv` de Pandas. Nota que indicamos que se use el punto y coma (";") como separador.

In [2]:
#Importa los datos del csv

df_classification = pd.read_csv('Earthworm_acute_toxicity_raw_data.csv', sep=';')

df_classification.head() #Muestra las primeras filas para comprobar que es el archivo adecuado

,SMILES,value,Experimental guidelines
0,COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1,Non-toxic,"OECD Guidelines 207: Earthworm, Acute Toxicity..."
1,CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O,Positive,"OECD Guidelines 207: Earthworm, Acute Toxicity..."
2,CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1,Non-toxic,"OECD Guidelines 207: Earthworm, Acute Toxicity..."
3,O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...,non-toxic,"OECD Guidelines 207: Earthworm, Acute Toxicity..."
4,O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...,negative,"OECD Guidelines 207: Earthworm, Acute Toxicity..."


In [3]:
# How many different experimental guidelines are there?
unique_values = df_classification['Experimental guidelines'].unique()
print(unique_values)

['OECD Guidelines 207: Earthworm, Acute Toxicity Tests'
 '850.3100 - Earthworm Subchronic Toxicity Test ']


In [4]:
# show number of lines in the dataset
print("Number of lines in the dataset:", len(df_classification))

Number of lines in the dataset: 202


TAREA 1.2 Obtén información del juego de datos

Primero, vamos a contar cuántas moléculas hay en el conjunto de datos. Una forma habitual de hacerlo es simplemente comprobar el tamaño de la tabla (usando el parámetro `shape` de un DataFrame, más detalles en https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html).

### **Q1.** ¿Cuántas moléculas hay en el juego de datos?

In [5]:
#Cuenta las moléculas

n_molecules = df_classification.shape[0]
print(f"Number of molecules(without filter by unique): {n_molecules}")


Number of molecules(without filter by unique): 202


Otra pregunta importante es: ¿qué columnas tiene? Imprime los nombres de las columnas.

In [6]:
#Mira el nombre de las columnas

df_classification.columns

Index(['SMILES', 'value', 'Experimental guidelines'], dtype='object')

Habrás visto que hay un campo en la base de datos relacionado con el protocolo experimental. Es importante revisar este tipo de información y evaluar qué opción es mejor: restringirse a un único método (esto reduce el ruido, pero disminuye la cantidad de datos) o combinar varios.

El primer paso es evaluar la distribución de los valores; la forma más rápida y sencilla de hacerlo es usando el método `value_counts` (https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html).

In [7]:
#Comprueba los protocolos experimentales

df_classification['Experimental guidelines'].value_counts()

Experimental guidelines
OECD Guidelines 207: Earthworm, Acute Toxicity Tests    174
850.3100 - Earthworm Subchronic Toxicity Test            28
Name: count, dtype: int64

En este caso, la base de datos incluye diferentes opciones, pero hay una mayoría significativa de valores correspondientes a un solo protocolo. Además, se trata de un protocolo estándar de la OCDE, lo que garantiza que estos valores se han obtenido siguiendo un método estandarizado y que su fiabilidad está establecida. Por tanto, en este caso restringiremos nuestro modelo a los datos obtenidos mediante este método.

Crea un nuevo DataFrame llamado `df_classification_filtered` que contenga únicamente los registros correspondientes a este método.

In [8]:
#Filtra la base de datos

#ESCRIBE AQUÍ TU CÓDIGO
df_classification_filtered = df_classification[df_classification['Experimental guidelines'] == 'OECD Guidelines 207: Earthworm, Acute Toxicity Tests']

#Muestra el número de filas al final, para comprobar que el filtro ha funcionado correctamente
df_classification_filtered.shape[0]

174

Ahora vamos a comprobar los valores del endpoint. Necesitamos revisar tres aspectos diferentes:

1. ¿Cómo están etiquetados los valores? ¿Ya tenemos una clasificación tal y como requiere este modelo? ¿Están las categorías etiquetadas de forma consistente?

1. ¿Tenemos valores vacíos, incorrectos o poco claros que debamos eliminar?

1. ¿Tenemos suficientes valores positivos y negativos? Para el modelado, el escenario ideal es contar con un conjunto de datos equilibrado.

Visualiza la distribución de los datos para el valor del endpoint (puedes usar el mismo enfoque que utilizamos para el protocolo experimental).

In [9]:
#Comprueba los valores usados para la toxicidad

df_classification_filtered['value'].value_counts()

value
negative     31
Negative     31
toxic        29
Non-toxic    24
Positive     23
positive     15
non-toxic    11
Toxic        10
Name: count, dtype: int64

En este caso, tenemos una variación significativa en las etiquetas. Por tanto, es necesario identificarlas de manera consistente.

Se recomienda usar valores numéricos (esto no es un requisito para todos los algoritmos, pero sí puede serlo para algunos, por lo que es mejor utilizar números de forma coherente). Usaremos 1 para los valores tóxicos o positivos y 0 para los valores no-tóxicos o negativos.

Convierte los valores a 1/0 y guárdalos en una columna llamada 'y' (que es un nombre habitual para el endpoint en aprendizaje automático).

Hay varias formas de hacerlo. Puedes preparar un diccionario con la equivalencia entre los diferentes valores y los valores 1 y 0, y luego usar el método `map` para convertir la columna '**value**' (https://pandas.pydata.org/docs/dev/reference/api/pandas.DataFrame.map.html). Alternativamente, puedes cambiar las etiquetas por números una por una utilizando el método `replace` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

In [10]:
#ESCRIBE AQUÍ TU CÓDIGO


df_classification_filtered['y'] = df_classification_filtered['value'].apply(lambda x: 1 if (x.lower() == 'toxic' or x.lower() == 'positive') else 0)

#Imprime tu Dataframe para comprobar que el progreso es correcto
df_classification_filtered


/var/folders/6m/y19m_stx56g4f1rh6ss7lkmw0000gn/T/ipykernel_59354/3785116728.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification_filtered['y'] = df_classification_filtered['value'].apply(lambda x: 1 if (x.lower() == 'toxic' or x.lower() == 'positive') else 0)


,SMILES,value,Experimental guidelines,y
0,COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1,Non-toxic,"OECD Guidelines 207: Earthworm, Acute Toxicity...",0
1,CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O,Positive,"OECD Guidelines 207: Earthworm, Acute Toxicity...",1
2,CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1,Non-toxic,"OECD Guidelines 207: Earthworm, Acute Toxicity...",0
3,O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...,non-toxic,"OECD Guidelines 207: Earthworm, Acute Toxicity...",0
4,O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...,negative,"OECD Guidelines 207: Earthworm, Acute Toxicity...",0
...,...,...,...,...
197,O=C(O)/C=C/C(=O)O.O=C(c1ccc(F)c(F)c1Nc1ccc(I)c...,positive,"OECD Guidelines 207: Earthworm, Acute Toxicity...",1
198,O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]...,positive,"OECD Guidelines 207: Earthworm, Acute Toxicity...",1
199,Cc1c(F)cc(C(=O)NC2CC2)cc1-c1ccc(C(=O)NCC(C)(C)...,positive,"OECD Guidelines 207: Earthworm, Acute Toxicity...",1
200,CClCCC=OF,negative,"OECD Guidelines 207: Earthworm, Acute Toxicity...",0


In [11]:
# Show y value counts to confirm the mapping
df_classification_filtered['y'].value_counts()

y
0    97
1    77
Name: count, dtype: int64

Una vez que el conjunto de datos está filtrado y listo, seleccionaremos únicamente las columnas de interés y lo guardaremos como un archivo .csv.

In [12]:
df_classification_final = df_classification_filtered[['SMILES', 'y']]

df_classification_final.to_csv('Earthworm_acute_toxicity-preprocessed.csv', sep=';', index=False)

## SECCIÓN 2. Limpiando las moléculas del juego de datos

Ahora que ya sabemos cómo importar los datos, vamos a comenzar a depurar el conjunto de datos para filtrar aquellos valores que no son correctos o que no son prácticos para su uso en el desarrollo de un modelo.

Trabajaremos con el conjunto de datos filtrado de toxicidad aguda en lombrices de tierra que creamos y guardamos en el archivo CSV "**Earthworm_acute_toxicity-preprocessed.csv**" en el paso anterior. Vamos a volver a cargar este conjunto de datos desde el archivo CSV.

Ten en cuenta que este paso no es estrictamente necesario, ya que el DataFrame aún está en memoria (acaba de guardarse). Sin embargo, se recomienda leerlo de nuevo, por si necesitas retomar tu trabajo más adelante o para poder repetir la **Sección 2**, asegurándote de que los datos iniciales sean consistentes entre ejecuciones.

In [13]:
#ESCRIBE AQUÍ TU CÓDIGO

df_classification = pd.read_csv('Earthworm_acute_toxicity-preprocessed.csv', sep=';')

#Visualiza el DataFrame para comprobar que es correcto
df_classification

,SMILES,y
0,COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1,0
1,CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O,1
2,CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1,0
3,O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...,0
4,O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...,0
...,...,...
169,O=C(O)/C=C/C(=O)O.O=C(c1ccc(F)c(F)c1Nc1ccc(I)c...,1
170,O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]...,1
171,Cc1c(F)cc(C(=O)NC2CC2)cc1-c1ccc(C(=O)NCC(C)(C)...,1
172,CClCCC=OF,0


### **Q2**. Vamos a comprobar que estamos utilizando el conjunto de datos adecuado. ¿Cuántas moléculas tienes en el conjunto de datos seleccionado en este punto?

In [14]:
#ESCRIBE AQUÍ TU CÓDIGO
# Count the number of unique SMILES
n_unique_smiles = df_classification['SMILES'].nunique()
print(f"Number of unique SMILES: {n_unique_smiles}")
n_lineas = df_classification.shape[0]
print(f"Number total de lineas: {n_lineas}") 

Number of unique SMILES: 168
Number total de lineas: 174


### 2.1. Limpiar los SMILES
Para empezar a trabajar con un conjunto de moléculas en formato SMILES, debemos verificar que el formato sea correcto. Este paso se denomina **saneamiento** o **sanitización**.

Antes de aplicarlo al conjunto de datos, podemos ver cómo funciona con un ejemplo sencillo. Por ejemplo, podemos comprobar si el **SMILES 'C(Cl)CCC=O'** representa una molécula válida. Lo convertiremos a un **objeto Mol** y lo volveremos a convertir a **SMILES** utilizando los métodos `MolFromSmiles` y `MolToSmiles` del módulo **Chem** de **RDKit** (http://rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html).

In [15]:
#Define el SMILES de la molécula que quieres comprobar
smi = 'C(Cl)CCC=O'

#Convierte el SMILES a un objeto mol
mol = Chem.MolFromSmiles(smi)

#Vuelve a obtener el SMILES del objeto mol
smi_sanitized = Chem.MolToSmiles(mol)

print('El SMILES {} es correcto y ha sido sanitizado a {}'.format(smi, smi_sanitized))

El SMILES C(Cl)CCC=O es correcto y ha sido sanitizado a O=CCCCCl


Si el SMILES de entrada es correcto, deberías ver un mensaje como: "**El SMILES C(Cl)CCC=O es correcto y ha sido sanitizado a O=CCCCCl**"

Observa que el SMILES devuelto no es exactamente el mismo que el original, pero si intentas convertirlo a una molécula, corresponde a la misma estructura.

Sin embargo, si el SMILES no es correcto, el método `MolFromSmiles` no puede devolver un objeto Mol y devuelve `None`. Por tanto, no podemos usar ese Mol para obtener un SMILES saneado. Intenta replicar el código que funcionó con '**C(Cl)CCC=O**' y aplícalo a '**CClC=CC=OF**'. Debería generar un error.

Ten en cuenta que, además del error de Python (incorrect type in the function), la salida incluye una advertencia de RDKit con detalles sobre el problema exacto en ese SMILES.

In [16]:
#Define el SMILES de la molécula que quieres comprobar
smi2 = 'CClC=CC=OF'


#Conviértelo a un objeto mol
mol = Chem.MolFromSmiles(smi2)
print(f"la molécula {smi2} fue convertida al objeto mol: {mol}")
#Vuelve a obtener el SMILES del objeto mol
print("Intentando sanitizar el SMILES...")
try:
    smi_sanitized = Chem.MolToSmiles(mol)
except Exception as e:
    print(f"Error sanitizing SMILES {smi2}: {e}")

la molécula CClC=CC=OF fue convertida al objeto mol: None
Intentando sanitizar el SMILES...
Error sanitizing SMILES CClC=CC=OF: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False, bool ignoreAtomMapNumbers=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)


[12:07:40] Explicit valence for atom # 1 Cl, 2, is greater than permitted


Este comportamiento es útil para detectar SMILES incorrectos, pero los errores que interrumpen la ejecución del código pueden ser un problema. Por tanto, una alternativa interesante es añadir una estructura `try/except` para evitar que el error detenga el código de forma abrupta, y devolver "None" en su lugar (lo que nos permite controlar qué ocurre en caso de encontrar un SMILES no adecuado).

This behavior is practical for detecting incorrect SMILES, but errors that interrupt the code are an issue. Thus, an interesting alternative is to add a try/except expression to bypass the error (so the code does not abruptly stop) and return `None` instead (allowing us to control what happens in case of inadequate SMILES).

In [17]:
#Define el SMILES de la molécula que quieres comprobar
smi2 = 'CClCCC=OF'



try:
  #Conviértelo a un objeto mol y vuelve a obtener el SMILES
  #Puede ser hecho en una sola línea.
    sanitized_smi2 = Chem.MolToSmiles(Chem.MolFromSmiles(smi2))
except:
    sanitized_smi2 = None

print(sanitized_smi2)

None


[12:07:40] Explicit valence for atom # 1 Cl, 2, is greater than permitted


Ahora vamos a aplicar esta técnica al DataFrame para nuestro modelo.
Itera sobre los elementos de la columna de SMILES y crea una lista llamada `new_smiles` con los SMILES saneados, y un valor vacío `None` para aquellos que sean incorrectos.

Después, utiliza esta lista para crear una nueva columna y filtra esa columna utilizando, por ejemplo, el método `notna` de pandas (https://pandas.pydata.org/docs/reference/api/pandas.notna.html).

Tu DataFrame resultante debería tener **166** moléculas.

In [18]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  # Desactiva los mensajes de advertencia de RDKit
#Crea una lista vacía para incluir los SMILES sanitizados
new_smiles= []
#Itera sobre la columna de SMILES
for smi in df_classification['SMILES']:
    try:
        #Sanitiza el SMILES si es posible
        sanitized_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi))
    except:
        #Usa None si no es posible (None sin "" porque es un valor vacío no un texto)
        sanitized_smi = None
    #Añade el valor a la lista new_smiles
    new_smiles.append(sanitized_smi)

#Incluye la lista new_smiles como una nueva columna 'san_SMILES'
df_classification['san_SMILES']= new_smiles

# filtra el DataFrame para eliminar los SMILES con None
df_classification = df_classification[df_classification['san_SMILES'].notnull()]


df_classification.shape

(166, 3)

### 2.2 Eliminar sales del DataFrame

Necesitamos eliminar las sales de nuestras moléculas, ya que entrenamos nuestros modelos sobre especies individuales. El primer paso es simplificar las sales que incluyen contraiones muy comunes, eliminando dichos contraiones y conservando el ion principal.

Consulta la documentación de RDKit para ver cómo hacer esto: https://www.rdkit.org/docs/source/rdkit.Chem.html.

In [19]:
#Importa la función SaltRemover de rdkit
from rdkit.Chem.SaltRemover import SaltRemover

#Lista de SMILES para analizar
list_of_SMILES = ['N#Cc1c(Cl)cccc1Cl.[Na+]',
'CC(C)(C)C(O)C(Oc1ccc(-c2ccccc2)cc1)n1cncn1.[Cl-]',
'CC(C)N(C(=O)SCC(Cl)=C(Cl)Cl)C(C)C.[K+]',
'CC(C)Oc1cc(-n2nc(C(C)(C)C)oc2=O)c(Cl)cc1Cl.[OH-]',
'Clc1ccc(C2(Cn3cncn3)CC(Br)CO2)c(Cl)c1.[NH3+]',
'C=CCOC(Cn1ccnc1)c1ccc(Cl)cc1Cl.[I-]',
'CCOc1nc(C(Cl)(Cl)Cl)ns1.[F-]']

#Itera sobre la lista de SMILES
for smi in list_of_SMILES:
    #Crea un objeto mol a partir del SMILES
    mol = Chem.MolFromSmiles(smi)
    #Inicializa el objeto SaltRemover con la lista de sales
    remover = SaltRemover(defnData='[Na,Cl,K,O,OH,Fe,F,H,Al,Mg,Co,Ti,NH4,Mn,Si,Ca,Au,I,Hg,Mo,Zn,Br,Ag,Sr,Cu,Bi,S,Li,NH3,He,Y,Ar,Ba,La]')
    #Aplica el remover sobre el objeto mol usando el método StripMol
    res = remover.StripMol(mol, dontRemoveEverything=True)
    #Crea el nuevo SMILES a partir del objeto mol sin sales (resultado del paso anterior)
    new_smi = Chem.MolToSmiles(res)
    #Muestra en pantalla el nuevo SMILES para comprobar el resultado
    print(new_smi)

N#Cc1c(Cl)cccc1Cl
CC(C)(C)C(O)C(Oc1ccc(-c2ccccc2)cc1)n1cncn1
CC(C)N(C(=O)SCC(Cl)=C(Cl)Cl)C(C)C
CC(C)Oc1cc(-n2nc(C(C)(C)C)oc2=O)c(Cl)cc1Cl
Clc1ccc(C2(Cn3cncn3)CC(Br)CO2)c(Cl)c1
C=CCOC(Cn1ccnc1)c1ccc(Cl)cc1Cl
CCOc1nc(C(Cl)(Cl)Cl)ns1


Aplica lo que hemos aprendido hasta ahora para eliminar las sales de nuestro DataFrame `df_classification`.

Para facilitar la comparación de los resultados, por favor guarda los SMILES sin sales en una nueva columna llamada '**NO SALTS**'.

In [20]:
from rdkit.Chem.SaltRemover import SaltRemover

smi_no_salts = []
for smi in df_classification['san_SMILES']:
    mol = Chem.MolFromSmiles(smi)
    remover = SaltRemover(defnData='[Na,Cl,K,O,OH,Fe,F,H,Al,Mg,Co,Ti,NH4,Mn,Si,Ca,Au,I,Hg,Mo,Zn,Br,Ag,Sr,Cu,Bi,S,Li,NH3,He,Y,Ar,Ba,La]')
    res = remover.StripMol(mol, dontRemoveEverything=True)
    new_smi = Chem.MolToSmiles(res)
    smi_no_salts.append(new_smi)

df_classification['NO SALTS'] = smi_no_salts


/var/folders/6m/y19m_stx56g4f1rh6ss7lkmw0000gn/T/ipykernel_59354/2236891574.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification['NO SALTS'] = smi_no_salts


Antes de continuar, habrás notado varias advertencias relacionadas con átomos de hidrógeno sin vecinos. Esto se debe a que el eliminador de sales no elimina protones aislados. Por tanto, sería una buena práctica eliminarlos utilizando otro enfoque (como detectar el fragmento **[H+]** en los SMILES). Para simplificar, no lo haremos en este curso.

Ten en cuenta que llamamos a esta parte **"eliminación de sales"**, pero esto no significa que eliminemos las sales del conjunto de datos; en realidad, lo que hacemos es simplificar sus SMILES para conservar únicamente el ion principal. Esto contrasta con otros pasos posteriores, como la eliminación de inorgánicos o mezclas, que sí eliminan datos del conjunto.

Sin embargo, puede ocurrir que una sustancia deba eliminarse en este proceso si está formada únicamente por los iones comunes que hemos quitado. Por ello, eliminaremos del conjunto de datos todos los casos en los que la columna "**NO SALTS**" haya quedado vacía.

¿Se ha eliminado alguna sustancia? (Puedes comprobarlo imprimiendo la longitud del conjunto de datos antes y después.)

In [21]:
# df_classification = df_classification[df_classification['NO SALTS']!= '']

# Print only df_classification lines where NO SALTS is different from san_SMILES
filtered_df = df_classification[df_classification['san_SMILES'] != df_classification['NO SALTS']]
print(filtered_df[['san_SMILES', 'NO SALTS']])


                                            san_SMILES  \
19    CCCCCCCCCCCCC1=C(OC(C)=O)C(=O)c2ccccc2C1=O.[Na+]   
29   Nc1c([N+](=O)[O-])ccc(Oc2ccccc2)c1Cl.[Cl-].[Cl...   
34    CC(C)(C)C(O)C(Oc1ccc(-c2ccccc2)cc1)n1cncn1.[Na+]   
35   CCOP(=S)(OCC)SCn1c(=O)oc2cc(Cl)ccc21.[Cl-].[Cl...   
54              CNC(=O)ON=C(C)SC.[Cl-].[Cl-].[H+].[H+]   
67    COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1.[Cl-].[H+]   
96   COC(=O)c1ccccc1S(=O)(=O)NC(=O)Nc1nc(C)nc(OC)n1...   
143              CCCCCCCCc1c(CC)nc2ncnn2c1N.[Cl-].[H+]   
153              CCCCCCCCc1c(CC)nc2ncnn2c1N.[Cl-].[H+]   

                                           NO SALTS  
19       CCCCCCCCCCCCC1=C(OC(C)=O)C(=O)c2ccccc2C1=O  
29   Nc1c([N+](=O)[O-])ccc(Oc2ccccc2)c1Cl.[H+].[H+]  
34       CC(C)(C)C(O)C(Oc1ccc(-c2ccccc2)cc1)n1cncn1  
35   CCOP(=S)(OCC)SCn1c(=O)oc2cc(Cl)ccc21.[H+].[H+]  
54                       CNC(=O)ON=C(C)SC.[H+].[H+]  
67       COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1.[H+]  
96   COC(=O)c1ccccc1S(=O)(=O)NC(=O)Nc1nc(

### 2.3 Eliminar moléculas inorgánicas y metalorgánicas

El siguiente paso es eliminar las sustancias inorgánicas y organometálicas. Para ello, definiremos una lista de átomos permitidos y eliminaremos cualquier sustancia que contenga otros átomos. Determinarás qué elementos están presentes en cada sustancia e iterarás sobre ellos para compararlos con la lista. Si algún elemento no está en la lista, eliminaremos la sustancia.

En este curso, permitiremos los siguientes elementos: **H, N, C, O, P, S, Si, F, Cl, Br e I**.

Explora esto en el ejemplo a continuación (dos sustancias deberían mantenerse y dos deberían eliminarse).

In [22]:
#Lista de SMILES para analizar
list_of_SMILES = ['Cl[Fe](Cl)Cl',
'CC(C)(C)C(O)C(Oc1ccc(-c2ccccc2)cc1)n1cncn1',
'[N+](=O)([O-])[O-].[N+](=O)([O-])[O-].[Cu+2]',
'O=S(=O)(O)O']

#Lista de átomos permitidos
allowed_atoms = ['H', 'N', 'C', 'O', 'P', 'S', 'Si', 'P', 'F', 'Cl', 'Br', 'I']

#Itera sobre la lista de SMILES
for smi in list_of_SMILES:
    #Crea un objeto mol a partir del SMILES
    mol = Chem.MolFromSmiles(smi)
    #Itera sobre los átomos de la molécula usando GetAtoms()
    for atom in mol.GetAtoms():
        #Obténe el símbolo del átomo usando GetSymbol()
        atom_symbol = atom.GetSymbol()
        #Comprueba si el átomo está en la lista de átomos permitidos
        if atom_symbol not in allowed_atoms:
           #Si no es un átomo permitido, elimina la molécula de la lista
           list_of_SMILES.remove(smi)
           break

#Comprueba los SMILES restantes
print(list_of_SMILES)

['CC(C)(C)C(O)C(Oc1ccc(-c2ccccc2)cc1)n1cncn1', 'O=S(=O)(O)O']


Una vez que consideres que tu código funciona correctamente, aplícalo a la base de datos.

Itera sobre los elementos de la columna "**NO SALTS**". Para eliminar las filas con átomos no permitidos, una forma práctica es crear primero una lista, por ejemplo llamada "**non_allowed_indx**", que contenga los índices de las filas cuyos SMILES contengan átomos no permitidos.

Después, puedes usar esta lista para eliminar esos índices del DataFrame utilizando el método `drop`. Esto no cambiará los índices, por lo que, si deseas tener un DataFrame ordenado con un índice que corresponda con las filas, se recomienda reiniciar el índice.

Alternativamente, puedes usar un enfoque similar al anterior y modificar los SMILES en la columna "**NO SALTS**" asignándoles un valor vacío.

In [25]:
allowed_atoms = ['H', 'N', 'C', 'O', 'P', 'S', 'Si', 'F', 'Cl', 'Br', 'I']

non_allowed_indx = []
#Reseteamos el índice para asegurarnos de que el índice corresponde a la posición real de los SMILES en la lista
df_classification.reset_index(drop=True, inplace=True)

#Usa enumerate para obtener el índice además del SMILES
for i, smi in enumerate(df_classification['NO SALTS']):

    mol = Chem.MolFromSmiles(smi)
    for atom in mol.GetAtoms():
        atom_symbol = atom.GetSymbol()
        if atom_symbol not in allowed_atoms:
            print(f"Molécula {smi} contiene un átomo no permitido: {atom_symbol}")
            #Añade el índice a la lista
            non_allowed_indx.append(i)
            break

df_classification.drop(non_allowed_indx,inplace=True)
df_classification.reset_index(drop=True, inplace=True)

df_classification


/var/folders/6m/y19m_stx56g4f1rh6ss7lkmw0000gn/T/ipykernel_59354/715795578.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification.drop(non_allowed_indx,inplace=True)


,SMILES,y,san_SMILES,NO SALTS
0,COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1,0,COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1,COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1
1,CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O,1,CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O,CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O
2,CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1,0,CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1,CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1
3,O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...,0,O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...,O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...
4,O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...,0,O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...,O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...
...,...,...,...,...
157,CCC(=O)O[C@]1(C(=O)CCl)[C@@H](C)C[C@H]2[C@@H]3...,1,CCC(=O)O[C@]1(C(=O)CCl)[C@@H](C)C[C@H]2[C@@H]3...,CCC(=O)O[C@]1(C(=O)CCl)[C@@H](C)C[C@H]2[C@@H]3...
158,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21,1,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21
159,O=C(O)/C=C/C(=O)O.O=C(c1ccc(F)c(F)c1Nc1ccc(I)c...,1,O=C(O)/C=C/C(=O)O.O=C(c1ccc(F)c(F)c1Nc1ccc(I)c...,O=C(O)/C=C/C(=O)O.O=C(c1ccc(F)c(F)c1Nc1ccc(I)c...
160,O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]...,1,O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]...,O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]...


### 2.4 Eliminar mezclas

Las **mezclas** están codificadas en **SMILES** mediante un punto que **separa** las dos moléculas, por ejemplo, **"CCO.O"** (**etanol y agua**). Para calcular descriptores, es importante que los **SMILES** representen únicamente **moléculas independientes**. Por lo tanto, eliminaremos las **mezclas** en este paso (esto también incluye la eliminación de cualquier sal que haya quedado en el paso anterior si contiene más de un componente).

Un buen enfoque sería explorar **manualmente** las mezclas para detectar el componente principal y conservarlo. **Por ejemplo**, a menudo es posible eliminar excipientes comunes como el **agua ('O'), el etanol ('CCO')** o el **DMSO ('CS(=O)C')**. Sin embargo, para simplificar el proceso en este curso, eliminaremos **todas** las mezclas.

**Por favor, elimina todas las mezclas** del DataFrame `df_classification`. Una forma sencilla es utilizar el carácter **"."** en los **SMILES** como indicador de mezcla.

In [26]:
mixtures_indx = []

for i, smi in enumerate(df_classification['NO SALTS']):
    if '.' in smi:
        print(f"Molécula {smi} es una mezcla")
        mixtures_indx.append(i)

df_classification.drop(mixtures_indx, inplace=True)
df_classification.reset_index(drop=True, inplace=True)

Molécula Nc1c([N+](=O)[O-])ccc(Oc2ccccc2)c1Cl.[H+].[H+] es una mezcla
Molécula CCOP(=S)(OCC)SCn1c(=O)oc2cc(Cl)ccc21.[H+].[H+] es una mezcla
Molécula CCC(=O)Nc1ccc(Cl)c(Cl)c1.O=S(=O)(O)O es una mezcla
Molécula COc1cc(OC)nc(NC(=O)NS(=O)(=O)N(C)S(C)(=O)=O)n1.O=C(O)/C=C/C(=O)O es una mezcla
Molécula CSc1nnc(C(C)(C)C)c(=O)n1N.O=S(=O)(O)O es una mezcla
Molécula CNC(=O)ON=C(C)SC.[H+].[H+] es una mezcla
Molécula COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1.[H+] es una mezcla
Molécula O=C(Nc1ccccc1)N(Cc1ccc(Cl)cc1)C1CCCC1.O=S(=O)(O)O es una mezcla
Molécula CCOc1nc(NC)nc(NC(=O)NS(=O)(=O)c2ccccc2C(=O)OC)n1.CS(C)=O es una mezcla
Molécula CCCCCCCCc1c(CC)nc2ncnn2c1N.[H+] es una mezcla
Molécula CCCCCCCCc1c(CC)nc2ncnn2c1N.[H+] es una mezcla
Molécula Cc1cc(F)ccc1-c1nc(NC(CO)CO)nc2c1ccc(=O)n2-c1c(F)cccc1F.Cc1ccc(S(=O)(=O)O)cc1 es una mezcla
Molécula Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21.O=S(=O)(O)O es una mezcla
Molécula O=C(O)/C=C/C(=O)O.O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]2CCCCN2)C1.O=C(c1

/var/folders/6m/y19m_stx56g4f1rh6ss7lkmw0000gn/T/ipykernel_59354/1011347772.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification.drop(mixtures_indx, inplace=True)


### 2.5 Eliminar duplicados

Para eliminar duplicados, Pandas ofrece una función integrada que realiza esta tarea.
Para obtener más información sobre cómo funciona esta función, puedes visitar:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

Los principales parámetros de esta función son:

* `subset`: columna (o columnas) que se deben considerar para identificar duplicados.

* `keep`: determina si se desea conservar el primer (`first`) o el último (`last`) duplicado (o ninguno si se establece en `False`).

**Nota**: Si decidimos conservar una de las moléculas duplicadas, primero debemos comprobar si tienen la misma variable de salida. Conservar duplicados con etiquetas conflictivas podría introducir sesgos e inconsistencias en nuestro conjunto de datos. Para evitar este problema y simplificar el proceso, en este caso eliminaremos todas las moléculas duplicadas.

In [30]:
df_classification.drop_duplicates(subset=['NO SALTS'], keep=False, inplace=True)

/var/folders/6m/y19m_stx56g4f1rh6ss7lkmw0000gn/T/ipykernel_59354/3212529102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classification.drop_duplicates(subset=['NO SALTS'], keep=False, inplace=True)


### **Q3.** Hagamos una comprobación final. ¿Cuántas moléculas tienes en tu conjunto de datos depurado?


In [31]:
print(f"Numero de moléculas después de eliminar mezclas y duplicados: {df_classification.shape[0]}")

Numero de moléculas después de eliminar mezclas y duplicados: 130


### 2.6 Selecciona y renombre las columnas para crear el juego de datos final

Con los **SMILES** **saneados** y la lista de **compuestos** depurada de sales, duplicados, etc., es momento de crear un conjunto de datos **final** con la lista definitiva de **SMILES** y sus valores de respuesta.

Es importante guardar la base de datos **saneada** como un archivo para poder utilizarla en los siguientes pasos del proceso y tener una copia que documente tu modelo.

In [41]:
#Crea un dataframe df_classification_final con las columnas 'NO SALTS' y 'y'
df_classification_final = df_classification[['NO SALTS', 'y']]

#Renombra las columnas como 'SMILES' y 'y'
df_classification_final.columns = ['SMILES', 'y']

# #Guarda el dataframe resultante en un archivo csv llamado Earthworm_acute_toxicity.csv
df_classification_final.to_csv('Earthworm_acute_toxicity.csv', sep=';', index=False)

0           COc1nc(C)nc(NC(=O)NS(=O)(=O)c2ccccc2OCCCl)n1
1            CCO/N=C(\CC)C1=C(O)CC(c2c(C)cc(C)cc2C)CC1=O
2                     CC1(C(=O)Nc2ccc(O)c(Cl)c2Cl)CCCCC1
3      O=C(NC(=O)c1c(F)cccc1F)Nc1cc(Cl)c(OC(F)(F)C(F)...
4      O=[N+]([O-])c1cc(C(F)(F)F)c(Cl)c([N+](=O)[O-])...
                             ...                        
143    CC(=O)O[C@H]1C[C@@H](C)[C@]23OC(C)(C)[C@H](C[C...
144    CCC(=O)O[C@]1(C(=O)CCl)[C@@H](C)C[C@H]2[C@@H]3...
145       Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21
146    O=C(c1ccc(F)c(F)c1Nc1ccc(I)cc1F)N1CC(O)([C@@H]...
147    Cc1c(F)cc(C(=O)NC2CC2)cc1-c1ccc(C(=O)NCC(C)(C)...
Name: NO SALTS, Length: 130, dtype: object